이번 연습은 **하지 말아야 할 예제**다!

# 충격적인 사례: 정크푸드와 여드름

정크푸드 섭취와 여드름 사이의 연관성을 조사하라는 요청을 받았다.

주어진 데이터셋은 `junkfood.csv`이며, 여러 열을 포함한다:

1. 0~10 사이의 **자가 보고 여드름 심각도** 숫자 열(0 = 전혀 없음, 10 = 항상 심함)
2. **주요 정크푸드 종류** 열(예: 초콜릿, 탄산음료 등)
3. **월별 정크푸드 섭취 횟수 추정** 열


In [11]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("junkfood.csv")
df.head()

,Acne,Food,Frequency
0,5,Burgers,21
1,6,Chocolate,26
2,5,Ice cream,17
3,1,Ice cream,1
4,0,Cake,2


Let's check for a global link

In [12]:
df.corr()

,Acne,Frequency
Acne,1.000000,0.018163
Frequency,0.018163,1.000000


정크푸드 섭취 빈도와 심한 여드름 사이에 **약한 양의 상관관계**가 있어 보인다.
이제 극단적인 사례를 비교하기 위해 일부 데이터를 살펴보자.


In [13]:
print(df.loc[df["Acne"] < 3, "Frequency"].mean())
print(df.loc[df["Acne"] > 7, "Frequency"].mean())

14.5357833655706
14.878205128205128


관계가 약해 보인다.

이제 귀무가설을 좀 더 명확히 정리하자.

**귀무가설(Null hypothesis):** *정크푸드 섭취 빈도는 여드름과 관련이 없다(상관이 없다).*

이를 검정하기 위해 **scipy의 Pearson 상관검정(Pearson correlation test)**을 사용할 수 있다.


In [14]:
from scipy.stats import pearsonr
correlation, pvalue = pearsonr(df["Acne"], df["Frequency"])
print(f"Correlation is {correlation:.3f}, with p-value of {pvalue:0.4f}")

Correlation is 0.018, with p-value of 0.5662


유의미한 p-value가 아니므로, 귀무가설을 기각할 수 없다.

하지만 한 가지 음식 종류별로 상관이 있는지 확인하면 어떨까? 각 음식 항목별로 검토해 보자.


In [18]:
foods = df["Food"].unique()
alpha = 0.05
for f in foods:
    df2 = df.loc[df["Food"] == f]
    correlation, pvalue = pearsonr(df2["Acne"], df2["Frequency"])
    if pvalue < alpha:
        print(f"Significant correlation (pvalue {pvalue:0.4f}) between {f}: {correlation:.3f}!!")
    else:
        print(f"No significant correlation for {f}")

No significant correlation for Burgers
No significant correlation for Chocolate
No significant correlation for Ice cream
Significant correlation (pvalue 0.0326) between Cake: 0.229!!
No significant correlation for Donuts
No significant correlation for Lollies
No significant correlation for Pure Sugar
No significant correlation for Cheese Pizza
No significant correlation for Brownies
No significant correlation for Milkshakes
No significant correlation for Soft Drink


**젠장, 케이크!**

이건 상상할 수 없는 일이다. 세상은 이 충격적인 소식을 어떻게 받아들일까?

---

이것은 **유의성 사냥(significance hunting)**의 전형적인 예이다. 초기 분석에서 결과가 없을 때 데이터를 파고들고, 많은 가설을 비교하면 결국 통계적으로 유의미한 결과를 발견하게 된다. 이 경우, 데이터는 원래 상관관계 없이 생성되었으므로 단순한 통계적 변동일 뿐이다. 하지만 결과를 찾기 위해, 케이크와 여드름 사이에 통계적으로 유의미한 상관관계가 있다고 주장하는 기사를 내고, 언론이 이를 보도하면, 모두 허위 과학(junk science)이 된다.

**교훈:** $\alpha$ 값(여기서는 0.05)은 검정 횟수를 고려하여 **거짓 양성(false positive)을 최소화**하도록 선택해야 한다.

또한, 다음과 같이 언론에서 상관관계가 있다고 보도할 때 주의해야 한다. 초콜릿과 심장병, 와인과 장수 등, 통계적으로 유의미한 상관이 있다고 해서 **상관의 크기** 자체가 의미 있는 것은 아니다.

충분히 큰 데이터 샘플이 있다면, 0.0001의 상관도 통계적으로 유의미하게 검출될 수 있다.

마지막으로 재미있는 예시로, [이 xkcd 만화](https://xkcd.com/882/)가 이번 여드름 예제의 영감을 주었다.
